In [10]:
!pip install tensorflow tensorflow-datasets --quiet

In [11]:
import tensorflow as tf
import tensorflow_datasets as tfds
from google.colab import files

In [12]:
ds_train, ds_test = tfds.load('beans', split=['train', 'test'], as_supervised=True)

IMG_SIZE = 128

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE)) / 255.0
    return image, label

ds_train = ds_train.map(preprocess).batch(32).prefetch(1)
ds_test = ds_test.map(preprocess).batch(32).prefetch(1)

In [13]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.Conv2D(16, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 classes in Beans dataset
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 126, 126, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 63, 63, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 61, 61, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 32)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,187 (20.26 KB)

 Trainable params: 5,187 (20.26 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
history = model.fit(ds_train, epochs=5, validation_data=ds_test)

Epoch 1/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 18s 469ms/step - accuracy: 0.3626 - loss: 1.0939 - val_accuracy: 0.5234 - val_loss: 1.0654
Epoch 2/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 11s 329ms/step - accuracy: 0.4643 - loss: 1.0570 - val_accuracy: 0.4375 - val_loss: 1.0317
Epoch 3/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 21s 344ms/step - accuracy: 0.4711 - loss: 1.0233 - val_accuracy: 0.4531 - val_loss: 1.0083
Epoch 4/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 11s 344ms/step - accuracy: 0.5123 - loss: 0.9876 - val_accuracy: 0.5625 - val_loss: 0.9565
Epoch 5/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 19s 302ms/step - accuracy: 0.5443 - loss: 0.9527 - val_accuracy: 0.5938 - val_loss: 0.9198


In [15]:
loss, accuracy = model.evaluate(ds_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 178ms/step - accuracy: 0.6292 - loss: 0.9069
Test Accuracy: 59.38%


In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('leaf_classifier.tflite', 'wb') as f:
    f.write(tflite_model)

files.download('leaf_classifier.tflite')

Saved artifact at '/tmp/tmphm1eeukq'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor_30')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  133991308339984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133991308340368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133991308339408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133991308341712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133991308339216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133991308343824: TensorSpec(shape=(), dtype=tf.resource, name=None)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>